The approach comes from [here]()

The only difference is, that I propose to substitute USE for SciBERT.

SciBERT can be accessed [here](https://github.com/allenai/scibert) via the [Huggingface transformers](https://github.com/huggingface/transformers) package.

Please investigate usage, I could not go into detail yet, sorry!

# Use SciBERT to vectorize text

__WARNING!__


Have to figure out if this last_hidden_states is a vector for a sentence a token or what. 


Requires checking the docs, I DID NOT TEST THIS!!!!

In [ ]:
!pip install transformers

In [ ]:
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

# Please check this usage, not sure it is ok!!!!!
input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]

# WARNING!
# Have to figure out if this last_hidden_states is a vector for a sentence a token or what. 
# Requires checking the docs, I DID NOT TEST THIS!!!!

In [ ]:
# If figured out, put the sentence vectors into sentence_vectors as numpy
sentence_vectors = embed(sentenced_text).numpy() #WARNING, this is a different approach, just illustration from a different vectorizer!

# Doing the distance matrix

If we have our nice sentence vectors, it is time to do a similarity / distance matrix from them, which we will feed to PageRank

In [ ]:
!pip install pandas
!pip install scikit-learn
!pip install networkx

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

distance_matrix = cosine_distances(sentence_vectors)

In [ ]:
import pandas as pd
df= pd.DataFrame(distance_matrix) #.stack()

stacked_df = df.stack()
stacked_df = stacked_df.rename_axis(('source', 'target')).reset_index(name='weight')
stacked_df["weight"]=stacked_df["weight"]*-1.0
stacked_df["weight"]=stacked_df["weight"]+2.0

# Calculating centrality

Now we have a nice distance matrix, time to run PageRank and see, which are the most "central" elements!

In [ ]:
import networkx as nx
G = nx.from_pandas_edgelist(stacked_df,  edge_attr=True)
print (nx.to_dict_of_dicts(G))

In [ ]:
pr = nx.pagerank(G, weight="weight", alpha=0.9)

# Alternative approach

We take the sentence vectors, and use them for approximate nearest neighbor searches, with the tool [Annoy](https://github.com/spotify/annoy). Basically this means, that if we can vectorize a query, we can use annoy to look up $n$ nearest neighbors, thus, we have a search engine.

Moreover, if we take the average vector of the document, use that as a query, we can get the most "important" sentences, that, which are the closest in meaning with the document itself.



In [ ]:
!pip install --user annoy --upgrade --force-reinstall

In [1]:
#WARNING: demo code below, the variable names are totally not the same, just copypasted...

In [ ]:
from annoy import AnnoyIndex

f = 768
t = AnnoyIndex(f, 'angular') 

for i in range(len(all_sentence_vectors)):
    t.add_item(i, all_sentence_vectors[i])

t.build(10)

nearest_sentence_indices = t.get_nns_by_vector(doc.vector/doc.vector_norm, 5)
sentences = list(doc.sents)

for i in nearest_sentence_indices:
    print(sentences[i])

In [ ]:
#WARNING: it is to be investigated, hogy SciBERT works, if the same as BERT, 
#then ALL the vectors are to be normalized!